# Data Analysis Migros

### Load libraries, load data, define static variables and set options.

In [1]:
# Import libraries.
import pandas as pd
import numpy as np
from shapely.geometry import MultiPoint, Point, Polygon, shape
from ast import literal_eval
import warnings; warnings.simplefilter('ignore')

# Load data.
supermarkets = pd.read_csv(
    'supermarket_coordinates_zurich.csv')[['name', 'lat', 'lon']]
sports = pd.read_csv(
    'stadiums_sportcenters_coordinates_zurich.csv')[['name', 'lat', 'lon']]
stations = pd.read_csv(
    'trains_coordinates_zurich.csv')[['name', 'lat', 'lon']]
school = pd.read_csv(
    'schools_buses_coordinates_zurich.csv')[['name', 'lat', 'lon']]
kreis_coord = pd.read_csv(
    'kreis_coordinates.csv')

# Set options.
pd.options.display.max_rows = 5000
pd.options.display.max_columns = 20

In [2]:
# More general method to define whether a coordinate is
# in the polygon or not.
def _isin_polygon(lat, lon, pol):
    
    # Define point.
    point = Point(lat, lon)
    
    return Polygon(pol).contains(point)

In [3]:
# Method to define whether a coordiante is within the 
# polygon of a given list of coordinates.
def isin_polygon(df, poly):
        
    # Empty list.
    res = []
    
    # Loop over all rows.
    for i, v in df.iterrows():
        res.append(_isin_polygon(v['lat'], v['lon'], poly))
    
    return res

In [4]:
# coords: a tuple containing coordinates ordered as (lat, lon).
def merc_proj(coords):

    lon, lat = coords    
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + \
        lat * (math.pi/180.0)/2.0)) * scale

    return x, y

In [ ]:
# More general method to buffer out the points by a 
# specific factor.
def _buffer_point(lat, lon, factor = 1.0):
    
    # Define point.
    point = Point(lat, lon)
    
    # Construc
    
    
    return 1

In [ ]:
# Method to buffer a point by a specific factor.
def buffer_point(df, factor):
        
    # Empty list.
    res = []
    
    # Loop over all rows.
    for i, v in df.iterrows():
        res.append(_buffer_point(v['lat'], v['lon'], factor))
    
    return res

In [5]:
# Add criteria to each data frame.
supermarkets['criteria'] = np.where(supermarkets['name'].str.contains(
    'Migros|Denner|Alnatura|VOI|Migrolino|M-Outlet'), 'Migros', 'market')
sports['criteria'] = 'sports'
stations['criteria'] = 'stations'
school['criteria'] = 'school_bus'

# Concat all to a single data frame.
data = pd.concat([supermarkets, sports, stations, school], 
                 ignore_index = True, axis = 0)

# Get an additional columns that indicate whether the stores are 
# located in which Kreis.
data['kreis'] = np.nan
for i, d in kreis_coord.iterrows():
    isin_res = isin_polygon(data[['lat', 'lon']], 
                                literal_eval(d['coords']))
    kreis_tmp = 'kreis_' + str(d['kreis'])
    data['kreis'][isin_res] = kreis_tmp

# Clean the stores by getting rid of "Top Market".
data = data[~data['name'].isin(['Top Market Oerlikon', 'no',
                                            'Minimum', 'Zürich'])]

# Buffer out the points to get circles around them.
data['circle'] = buffer_point(data[['lat', 'lon']], factor = 5)

# Save to csv.
#data.to_csv('data_clean.csv')
print(data[['name', 'criteria','kreis']])

                                                  name    criteria     kreis
0                                   Coop Bahnhofbrücke      market   kreis_1
1                                      Migros Wengihof      Migros   kreis_4
2                                                 Aldi      market   kreis_2
3                                     Coop Wollishofen      market   kreis_2
4                                   Migros Wollishofen      Migros   kreis_2
5                                          Migros City      Migros   kreis_1
6                                    Coop Hönggermarkt      market  kreis_10
7                                               Denner      Migros  kreis_10
8                                  Migros Bahnhof Enge      Migros   kreis_2
9                                        Coop Fluntern      market   kreis_7
10                               Coop Albisriederplatz      market   kreis_3
11                                         Aldi Suisse      market   kreis_9